In [1]:
# aspects: inhouse generation and purchased 

# inputs:
# inhouse generation = Wi(t)
# price of electricity = Ci(t)
# the total incoming load = L(t)
# the QoS parameter = q

# outpus:
# the allocated workload to each datacenter
# green workload, Ygi
# brown workload, Ybi
# total workload, Y


# Algorithm:
# i)    assume we have n datacenter
# ii)   assume we are optimizing for each datacenter at once, and time slot are divided in hours
# iii)  assume we have N division in the timeslot
# iv)   for each time slot we will calculate the Y

# Idea:
# i)    we will consider Wi(t) to be a constant or zero
# ii)   we will consider the price to be the announced per unit price of energy( electricity )
# iii)  incoming load can be the load of peek hour
# iv)   QoS parameter neglected


# Brown Energy Treatment and Termination Algorithm
# columns: 'time', 'total_incoming_green_energy', 'total_incoming_brown_energy',
    #    'total_incoming_load', 'total_actual_load', 'green_energy_price',
    #    'brown_energy_price'
class Better(object):
    def __init__(self, dataframe):
        # the replicated dataset
        self.df = dataframe.copy()

        # the columns in ndarray
        self.time = self.df['time'].values
        self.total_incoming_green_energy = self.df['total_incoming_green_energy'].values
        self.total_incoming_brown_energy = self.df['total_incoming_brown_energy'].values
        self.total_incoming_load = self.df['total_incoming_load'].values
        self.total_actual_load = self.df['total_actual_load'].values
        self.green_energy_price = self.df['green_energy_price'].values
        self.brown_energy_price = self.df['brown_energy_price'].values

        #iterators
        self.time_it = iter(self.time)
        self.total_incoming_green_energy_it = iter(self.total_incoming_green_energy)
        self.total_incoming_brown_energy_it = iter(self.total_incoming_brown_energy)
        self.total_incoming_load_it = iter(self.total_incoming_load)
        self.total_actual_load_it = iter(self.total_actual_load)
        self.green_energy_price_it = iter(self.green_energy_price)
        self.brown_energy_price_it = iter(self.brown_energy_price)
        
        # params
        self.batch_size = 24

    def lenth(self):
        return len(self.df)
    
    def __len__(self):
        return len(self.df)
    
    def days(self):
        return len(self.df)/self.batch_size
    
    def load_batch(self, batch_size, it):
        buffer = []
        for _ in range(batch_size):
            buffer.append(next(it))
        return buffer

    def evalute(self, days=None):
        if not days:
            days = self.days()
        
        for i in range(days):
            print(f'################################:::DAY-{i+1}:::################################')
            # green workload, Ygi
            Ygi = self.load_batch(batch_size=self.batch_size, it=self.total_incoming_green_energy_it)
            # brown workload, Ybi
            Ybi = self.load_batch(batch_size=self.batch_size, it=self.total_incoming_brown_energy_it)
            # actual load, L
            L = self.load_batch(batch_size=self.batch_size, it=self.total_actual_load_it)

            # per day savings
            SAVE = 0
            SAVE_G = 0
            SAVE_B = 0
            # total workload, Y (per day)
            Y = 0
            # for test
            L_cum = 0
            # per time slot
            for i, (Lgi, Lbi, Li) in enumerate(zip(Ygi, Ybi, L)):
                L_cum += Li
                # per hour savings
                save = 0
                save_g = 0
                save_b = 0
                # per hour workload
                yi = 0
                # we are doing in respect to the availability of green energy
                if Lgi<Li: # when green is less
                    if Lbi>Li: # utilize brown
                        yi += Li
                        save_b += Lbi-Li
                    else: # utilize brown and saved green
                        deficit = Li-Lbi
                        complement_g = SAVE_G - deficit
                        complement_b = SAVE_B - deficit 

                        complement_status_g = True if complement_g>=0 else False 
                        complement_status_b = True if complement_b>=0 else False 

                        match((complement_status_b, complement_status_g)):
                            case (False, False):
                                yi += Lbi # this can still occur deficit in total
                            case (False, True):
                                yi += Li
                                SAVE_G -= deficit
                                SAVE -= deficit

                            case (True, False):
                                yi += Li
                                SAVE_B -= deficit
                                SAVE -= deficit
                            case (True, True):
                                yi += Li
                                SAVE_G -= deficit
                                SAVE -= deficit

                else: # when green is not only fulfilling demand but adequate in amount
                    if Lgi > Li: # green is more
                        yi += Li
                        save_g += Lgi-Li
                    else: # green is same as demand
                        yi += Lgi
                save = save_g + save_b
                SAVE_G += save_g
                SAVE_B += save_b
                Y += yi
                # print(f'Hour {i+1}: Green Energy Saved: {save_g:.2f}, Brown Energy Saved:{save_b:.2f}')
            SAVE += SAVE_G + SAVE_B
            print('')
            print(f'Total Day workload: {Y:.2f} , Total Energy Saved: {SAVE:.2f}')
            print('')
            print(f'Total Green Energy Saved: {SAVE_G:.2f} , Total Brown Energy Saved: {SAVE_B:.2f}')
            print('')
            if L_cum == Y:
                print('Verdict: Fully Utilized!!!')
            else:
                print(f'Verdict: Deficiency!!!\n\t{L_cum-Y:.2f} energy deficit!')
            
            print('################################:::DAY-END:::################################')
                
                


In [2]:
# test 
import pandas as pd

df = pd.read_csv('preprocessed.csv')

# om
b = Better(dataframe=df)
d = int(b.days())
b.evalute(d)

################################:::DAY-1:::################################

Total Day workload: 150.08 , Total Energy Saved: 0.00

Total Green Energy Saved: 0.00 , Total Brown Energy Saved: 0.00

Verdict: Deficiency!!!
	250.05 energy deficit!
################################:::DAY-END:::################################
################################:::DAY-2:::################################

Total Day workload: 283.66 , Total Energy Saved: -11.96

Total Green Energy Saved: 4.43 , Total Brown Energy Saved: 0.00

Verdict: Deficiency!!!
	172.63 energy deficit!
################################:::DAY-END:::################################
################################:::DAY-3:::################################

Total Day workload: 238.35 , Total Energy Saved: -13.95

Total Green Energy Saved: 0.06 , Total Brown Energy Saved: 0.00

Verdict: Deficiency!!!
	182.11 energy deficit!
################################:::DAY-END:::################################
##############################